<a href="https://colab.research.google.com/github/Jahnvi99/qWiki-Search/blob/main/qWiki_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11697 sha256=52cecebb9e7f0a2849e8e6fb707bbcb9586ca831cafa727695036137269616e4
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


#Importing libraries

In [2]:
import re
import nltk
import numpy as np
import wikipedia
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Finding relevant wikipedia pages

In [15]:
query = input('Enter query: ')
n = int(input('Enter number of sentences to be returned: '))

titles = wikipedia.search(query, results=5)
if not titles:
	print ('The given query does not match any wikipedia page.')
	exit()

Enter query: Elon Musk
Enter number of sentences to be returned: 10


#Extracting temporal sentences based only on year

In [16]:
data = []
for title in titles:
    try:
        raw_data = wikipedia.page(title).content.replace('\n', ' ')
        processed_data = re.sub('== References ==.+|== See Also ==.+', '', raw_data) #removing irrelevant data from content
        data.append(processed_data)
    except Exception:
        pass

articles = [nltk.sent_tokenize(text) for text in data]
temporal_sentences = []
for article in articles:
    for sentence in article:
        if re.search(r'\d{4}', sentence) is not None:               
            temporal_sentences.append(re.sub('=+.+?=+', '', sentence)) #removing all headings and subheadings

#Creating the document word matrix A

In [17]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(temporal_sentences)

#Creating the model to calculate A=WH

In [18]:
nmf_model = NMF(n_components=1, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
W = nmf_model.transform(tfidf)

#Selecting top documents based on the score from topic to document matrix(W)

In [19]:
top_sent_indices = np.argsort( W[:,0] )[::-1][0:n]
best_sentences = [temporal_sentences[sent_index] for sent_index in top_sent_indices]

#Sort and print the final output




In [20]:
output = []
for line in best_sentences:
    year = re.search(r'\d{4}', line).group()
    output.append((year, line))
sorted_output = sorted(output, key=lambda tup: tup[0], reverse = True)
for item in sorted_output:
	print(item)

('2019', 'ChinaIn August 2019, Musk announced that he would be launching The Boring Company China during a trip at the end of the month.')
('2018', 'The Boring Company was initially formed as a subsidiary of SpaceX before becoming an independent company in 2018.In an interview during a TED conference in April 2017, Musk estimated that The Boring Company project had taken 2–3% of his time, making this venture a personal hobby.In March 2017, Musk announced that sometime in April the company would start using a tunnel boring machine (TBM) to begin digging a usable tunnel at SpaceX.')
('2018', 'The Boring Company was initially formed as a subsidiary of SpaceX, becoming a separate and fully independent company in 2018.')
('2018', "Subsequent concerns by SpaceX shareholders resulted in a December 2018 reallocation of 6% of The Boring Company's equity to SpaceX.TBC provided an update on the state of their technology and product line in December 2018 when they opened to the public their first 